In [1]:
import catboost as cb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import numpy as np
import random
import matplotlib.pyplot as plt 
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import os

### Data Import

In [2]:
def rename_col_name(poi_name, col):
    if col == "hdb_lat":
        return "LATITUDE"
    elif col == "hdb_lon":
        return "LONGITUDE"
    else:
        return f"{poi_name}_{col}"

inflation_rates = pd.read_csv("../datasets/hdb/inflation_rates.csv", index_col='year')
def calculate_inflation(price, year):
    return price * inflation_rates.loc[year].values[0]


In [3]:
def get_data(min_year, max_year, selected_features=[]):
    random.seed(42)
    
    # HDB transactions
    hdb_trans = pd.read_csv("../datasets/hdb/hdb.csv")
    hdb_trans['year'] = pd.to_datetime(hdb_trans['month']).dt.year
    hdb_trans = hdb_trans[(hdb_trans['year'] >= min_year) & (hdb_trans['year'] <= max_year)]
    hdb_trans['remaining_lease'] = hdb_trans['lease_commence_date'] + 99 - hdb_trans['year']

    # Calculate the inflation adjusted resale price
    hdb_trans['resale_price'] = hdb_trans.apply(lambda x: calculate_inflation(x['resale_price'], x['year']), axis=1)

    # Addresses to get the lat and lon of each transaction
    addresses = pd.read_csv("../datasets/hdb/addresses.csv")[['QUERY', 'LATITUDE', 'LONGITUDE']]
    addresses[['LATITUDE', 'LONGITUDE']] = addresses[['LATITUDE', 'LONGITUDE']].round(5)
    df = pd.merge(hdb_trans, addresses, left_on='address', right_on='QUERY', how='inner')

    # Merge the poi data
    for file in os.listdir("../datasets/poi/"):
        if "hdb" in file:
            # Get the poi name
            poi_name = file.split("_")[0]
            poi = pd.read_csv(f"../datasets/poi/{file}")

            # Drop unnecessary columns
            if 'Unnamed: 0' in poi.columns:
                poi = poi.drop(columns=['Unnamed: 0', f'{poi_name}_lat', f'{poi_name}_lon'])
            
            # Rename the columns to differentiate between the different poi
            poi.columns = [rename_col_name(poi_name, col) for col in poi.columns]
            
            # Round the lat and lon columns
            poi[['LATITUDE', 'LONGITUDE']] = poi[['LATITUDE', 'LONGITUDE']].round(5)

            # Merge the poi with the bus_counts
            df = pd.merge(df, poi, on=['LATITUDE', 'LONGITUDE'])

    # Drop the unnecessary columns
    X = df.drop(columns=['resale_price', 'QUERY', 'LATITUDE', 'LONGITUDE', 'month', 'address', 'year', 'town', 'flat_type', 'lease_commence_date'])
    y = df['resale_price']

    # Drop the columns with high VIF
    if selected_features:
        X = X[selected_features]
    
    return X, y

X, y = get_data(2023, 2023)

### Feature Selection

In [4]:
import statsmodels.api as sm

sel_X = X.copy()
sel_X = sm.add_constant(sel_X)
sel_X = sel_X.drop(columns=['bus_distance',
                            'bus_within_0.1',
                            'bus_within_0.3',
                            'bus_within_1',
                            'bus_within_1.5',
                            'bus_within_2.0',
                            'mrtlrt_within_0.1',
                            'mrtlrt_within_0.3',
                            'mrtlrt_within_0.5',
                            'mrtlrt_within_1',
                            'mrtlrt_within_1.5',
                            'mrtlrt_within_2.0',
                            'mrtlrt_within_2.0',
                            'hawker_within_0.1',
                            'hawker_within_0.3',
                            'hawker_within_0.5',
                            'hawker_within_1',
                            'hawker_within_1.5',
                            'hawker_within_2.0',
                            'park_within_0.1',
                            'park_within_0.3',
                            'park_within_0.5',
                            'park_within_1',
                            'park_within_1.5',
                            'park_within_2.0',
                            'school_distance',
                            'school_within_0.1',
                            'school_within_0.3',
                            'school_within_0.5',
                            'school_within_1',
                            'school_within_1.5',
                            'supermarket_within_0.1',
                            'supermarket_within_0.3',
                            'supermarket_distance',
                            'supermarket_within_1',
                            'supermarket_within_1.5',
                            'supermarket_within_2.0',
                            'mall_distance',
                            'mall_within_0.1',
                            'mall_within_0.3',
                            'mall_within_0.5',
                            'mall_within_1',
                            'mall_within_1.5',])

est = sm.OLS(y, sel_X)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:           resale_price   R-squared:                       0.813
Model:                            OLS   Adj. R-squared:                  0.813
Method:                 Least Squares   F-statistic:                 1.117e+04
Date:                Tue, 26 Mar 2024   Prob (F-statistic):               0.00
Time:                        17:36:36   Log-Likelihood:            -3.2469e+05
No. Observations:               25672   AIC:                         6.494e+05
Df Residuals:                   25661   BIC:                         6.495e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                  -1.07

In [5]:
vif_df = sel_X.copy().drop(columns=[])

# VIF dataframe 
vif_data = pd.DataFrame()
vif_data["feature"] = vif_df.columns 

# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(vif_df.values, i) 
                          for i in range(len(vif_df.columns))] 
  
print(sorted(vif_data.values, key=lambda x: x[1], reverse=True)[0:2])
print(vif_data)

[array(['const', 63.86820329694119], dtype=object), array(['cbd_distance', 1.5726501567707694], dtype=object)]
                   feature        VIF
0                    const  63.868203
1           floor_area_sqm   1.084644
2          remaining_lease   1.151178
3           bus_within_0.5   1.152116
4             cbd_distance   1.572650
5          hawker_distance   1.249053
6          mall_within_2.0   1.168380
7          mrtlrt_distance   1.168250
8            park_distance   1.208653
9        school_within_2.0   1.204286
10  supermarket_within_0.5   1.095550


In [6]:
selected_features = vif_data[vif_data['VIF'] < 5]['feature'].values.tolist()

### Catboost Model

In [7]:
# initialize grid
grid = {'iterations': [250, 500, 750],
        'depth': [10, 12, 14, 16],
        'l2_leaf_reg': [0.3, 0.5, 1]}

def train(min_year, max_year):
    print(f"Training model for year {min_year}-{max_year}")

    # Get the data
    X, y = get_data(min_year, max_year, selected_features)
    dataset = cb.Pool(X, y)
    model = cb.CatBoostRegressor(iterations=500,
                                  depth=16,
                                  l2_leaf_reg=0.3,
                                  loss_function="RMSE", 
                                  logging_level='Silent',
                                  task_type="GPU",
                                  devices='0')
    
    # Fit the model
    model.fit(dataset)
    
    # Evaluate the model
    pred = model.predict(X)
    mape = mean_absolute_percentage_error(y, pred)
    rmse = np.sqrt(mean_squared_error(y, pred))
    r2 = r2_score(y, pred)
    print(f"Testing performance for year {min_year}-{max_year}:")
    print(f'MAPE: {mape:.2%}')
    print(f'RMSE: {rmse:.2f}')
    print(f'R2: {r2:.2f}')

    # Save the feature importances
    features_df = model.get_feature_importance(prettified=True)
    features_df.to_csv(f"./feature_importances/catboost_{min_year}_{max_year}.csv")

In [8]:
for min_year in range(2000, 2024, 4):
    max_year = min_year + 3
    if min_year == 2020:
        max_year = max_year + 1
    
    train(min_year, max_year)

Training model for year 2000-2003


Testing performance for year 2000-2003:
MAPE: 4.79%
RMSE: 20994.00
R2: 0.98
Training model for year 2004-2007
Testing performance for year 2004-2007:
MAPE: 5.57%
RMSE: 26072.86
R2: 0.95
Training model for year 2008-2011
Testing performance for year 2008-2011:
MAPE: 5.34%
RMSE: 28440.16
R2: 0.95
Training model for year 2012-2015
Testing performance for year 2012-2015:
MAPE: 3.64%
RMSE: 21374.56
R2: 0.97
Training model for year 2016-2019
Testing performance for year 2016-2019:
MAPE: 4.37%
RMSE: 27777.38
R2: 0.97
Training model for year 2020-2024
Testing performance for year 2020-2024:
MAPE: 4.93%
RMSE: 35089.14
R2: 0.96
